<a href="https://colab.research.google.com/github/oneoclockc/deeplearning-for-AI/blob/main/chapter11_part01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning for text

## Natural-language processing: The bird's eye view

- text classification : 다음 text의 주제는?
- content filtering : 이 텍스트가 abuse를 포함하는가?
- sentiment analsis : 이 text가 positive / negative?
- laguage modeling : 다음 문장의 다음에 올 단어는?
- translation : 번역
- summarization : 요약

## Preparing text data

### Text standardization

  sunset came. I was satring at the Mexico sky. Isnt nature splendid?
- 소문자 변환, 문장부호 제거  
> sunset came i was staring at the mexico sky isnt nature splendid
- 용어의 변형(시제 등)을 단일 표현으로 변환  
> sunset came i was stare at the mexico sky isnt nature splendid

### Text splitting (tokenization)

- word-level tokenization : 텍스트를 단어로 나누고 각 단어를 하나의 벡터로 변환
- N-gram tokenization : N-gram은 n개의 연속 단어 그룹
- character-level tokenization : 텍스트를 문자로 나누고 각 문자를 하나의 벡터로 변환(거의 사용 x)

### Vocabulary indexing

생성된 토큰을 numeric vector 로 변경. one-hot encoding과 token imbedding이 대표적
- one-hot encoding : 모든 단어에 고유한 정수 인덱스를 부여, 이 정수 인덱스 i를 크기가 N(어휘사전의 크기)인 이진 벡터로 변환 후 i번째 원소는 1로, 나머지는 0으로 인코딩

### Using the TextVectorization layer

In [2]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower() # 소문자 변환 
        return "".join(char for char in text if char not in string.punctuation) # 구두점 제거 

    def tokenize(self, text): 
        text = self.standardize(text) # standardize 후 공백 기준으로 split
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1} # 1: out of vocabulary(i cant recognize it) / 0 : mask token(ignore it. im not a word!)
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary: # vocabulary에 없는 단어에 대해 등록
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [3]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence) # still -> cannot recognize

[2, 3, 5, 7, 1, 5, 6]


In [4]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [5]:
# 실제로는 keras의 textvectorization layer를 사용
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    output_mode="int", # 정수 단위로 단어를 인코딩 
)

In [6]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor) # 소문자 변환 
    return tf.strings.regex_replace( # 구두점 문자를 빈 문자열로 변환(제거)
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor): 
    return tf.strings.split(string_tensor) # 공백 기준으로 문장 분할

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [7]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset) # text corpus의 어휘를 인덱스화

**Displaying the vocabulary**

In [8]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [9]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence) # 인코딩된 문장 확인 

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [10]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence) # 문장으로 다시 디코딩 

i write rewrite and [UNK] rewrite again


## Two approaches for representing groups of words: Sets and sequences

bag-of-word model : 단어의 순서 고려 x  
RNN, transformers : sequence models

### Preparing the IMDB movie reviews data

In [11]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  15.1M      0  0:00:05  0:00:05 --:--:-- 19.0M


In [12]:
!rm -r aclImdb/train/unsup

In [13]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [14]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files) # 시드 설정 
    num_val_samples = int(0.2 * len(files)) # val set -> train set에서 20% 할당
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [15]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
) # dataset 만들기

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Displaying the shapes and dtypes of the first batch**

In [16]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Worry not, Disney fans--this special edition DVD of the beloved Cinderella won\'t turn into a pumpkin at the strike of midnight. One of the most enduring animated films of all time, the Disney-fide adaptation of the gory Brothers Grimm fairy tale became a classic in its own right, thanks to some memorable tunes (including "A Dream Is a Wish Your Heart Makes," "Bibbidi-Bobbidi-Boo," and the title song) and some endearingly cute comic relief. The famous slipper (click for larger image) We all know the story--the wicked stepmother and stepsisters simply won\'t have it, this uppity Cinderella thinking she\'s going to a ball designed to find the handsome prince an appropriate sweetheart, but perseverance, animal buddies, and a well-timed entrance by a fairy godmother make sure things turn out all right. There are a few striking sequences of pure animation--for exam

### Processing words as a set: The bag-of-words approach

#### Single words (unigrams) with binary encoding

**Preprocessing our datasets with a `TextVectorization` layer**

In [17]:
text_vectorization = TextVectorization(
    max_tokens=20000, # 어휘를 가장 빈도수가 높은 20000개로 제한
    output_mode="multi_hot", # multi hot binary vector로 인코딩 
)
text_only_train_ds = train_ds.map(lambda x, y: x) # raw text input만 생성하는 데이터 집합을 준비
text_vectorization.adapt(text_only_train_ds) # 데이터 집합 인덱싱

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), # processed dataset
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**Inspecting the output of our binary unigram dataset**

In [21]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


**Our model-building utility**

In [20]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x) 
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

**Training and testing the binary unigram model**

In [22]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(), # cache()를 통해 메모리에 저장, 데이터가 메모리에 들어갈 정도로 작은 경우에만 수행 가능 
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 13s 19ms/step - loss: 0.4150 - accuracy: 0.8273 - val_loss: 0.2956 - val_accuracy: 0.8840
Epoch 2/10
625/625 [=======================

#### Bigrams with binary encoding

**Configuring the `TextVectorization` layer to return bigrams**

In [23]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

**Training and testing the binary bigram model**

In [24]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 14s 21ms/step - loss: 0.3868 - accuracy: 0.8396 - val_loss: 0.2704 - val_accuracy: 0.8966
Epoch 2/10
625/625 [=====================

#### Bigrams with TF-IDF encoding

**Configuring the `TextVectorization` layer to return token counts**

In [25]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

**Configuring `TextVectorization` to return TF-IDF-weighted outputs**

In [26]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

**Training and testing the TF-IDF bigram model**

In [27]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 13s 20ms/step - loss: 0.4630 - accuracy: 0.7961 - val_loss: 0.3232 - val_accuracy: 0.8664
Epoch 2/10
625/625 [=====================

In [28]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [29]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

90.64 percent positive
